In [2]:
import numpy as np

In [91]:
# Initialization
pauli_z = np.array([[1,0],
                    [0,-1]])

pauli_x = np.array([[0,1],
                   [1,0]])

zero = np.zeros((2,2))

identity = np.identity(2)

g = 1

class Hamiltonian:
    # Matrix Product States
    # First lattice position (1x3x2x2)
    left_bound = np.array([identity, pauli_z, g*pauli_x])
    left_bound = left_bound[np.newaxis, :] # Reshape to add the first index
    
    # Middle lattice positions (3x3x2x2)
    middle = np.array([np.array([identity, pauli_z, g*pauli_x]),
                       np.array([zero, zero, pauli_z]),
                       np.array([zero, zero, np.identity(2)])])

    # Last lattice position (3x1x2x2)
    right_bound = np.array([[g*pauli_x],
                               [pauli_z],
                               [identity]])

In [92]:
H = Hamiltonian()

(1, 3, 2, 2)